In [1]:
import sys
import os

In [2]:
os.environ.get('JAVA_HOME')

'C:\\Program Files\\Java\\jdk1.8.0_311'

In [3]:
import findspark

findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder \
                    .master("local[*]") \
                    .config("spark.sql.warehouse.dir", "temp") \
                    .appName("SparkSQL") \
                    .getOrCreate()

In [6]:
data = spark.read.option("delimiter", "|").csv("data/ml-100k/u.item")

In [7]:
df = data.select(col("_c0").alias("id"), col("_c1").alias("name"))

In [8]:
df.show(10, truncate=False)

+---+----------------------------------------------------+
|id |name                                                |
+---+----------------------------------------------------+
|1  |Toy Story (1995)                                    |
|2  |GoldenEye (1995)                                    |
|3  |Four Rooms (1995)                                   |
|4  |Get Shorty (1995)                                   |
|5  |Copycat (1995)                                      |
|6  |Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|
|7  |Twelve Monkeys (1995)                               |
|8  |Babe (1995)                                         |
|9  |Dead Man Walking (1995)                             |
|10 |Richard III (1995)                                  |
+---+----------------------------------------------------+
only showing top 10 rows



In [9]:
df = df.union(df).union(df).union(df).union(df)

In [10]:
df.repartition(5)

DataFrame[id: string, name: string]

In [11]:
df.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|    Toy Story (1995)|
|  2|    GoldenEye (1995)|
|  3|   Four Rooms (1995)|
|  4|   Get Shorty (1995)|
|  5|      Copycat (1995)|
|  6|Shanghai Triad (Y...|
|  7|Twelve Monkeys (1...|
|  8|         Babe (1995)|
|  9|Dead Man Walking ...|
| 10|  Richard III (1995)|
| 11|Seven (Se7en) (1995)|
| 12|Usual Suspects, T...|
| 13|Mighty Aphrodite ...|
| 14|  Postino, Il (1994)|
| 15|Mr. Holland's Opu...|
| 16|French Twist (Gaz...|
| 17|From Dusk Till Da...|
| 18|White Balloon, Th...|
| 19|Antonia's Line (1...|
| 20|Angels and Insect...|
+---+--------------------+
only showing top 20 rows



## Get a DataFrame's Per-Partition Count

In [12]:
rcrd_cnt_part = df.withColumn("partitionid", spark_partition_id()) \
                  .groupBy("partitionid") \
                  .count()

In [13]:
rcrd_cnt_part.show()

+-----------+-----+
|partitionid|count|
+-----------+-----+
|          0| 1682|
|          1| 1682|
|          2| 1682|
|          3| 1682|
|          4| 1682|
+-----------+-----+



## WRITE MAX RECORDS PER FILE

## RESTRICT FILE SIZE (PART FILE)

Here we are saving the dataframe in CSV format. And also specified "maxPartitionBytes" as 3MB, which means the maximum size of each part-file is not more than 3MB.